In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
#READING INPUT
data = pd.read_csv("/kaggle/input/spooky-author-identification/train.zip")
data.head()

In [ ]:
data['author_num'] = data["author"].map({'EAP':0, 'HPL':1, 'MWS':2})
data.head()

In [ ]:
train=data.sample(frac=0.8,random_state=200) #random state is a seed value
test=data.drop(train.index)

In [ ]:
!pip install simpletransformers

In [ ]:
# import wandb

# wandb.login()

In [ ]:
# import logging

# import pandas as pd
# import sklearn

# import wandb
# from simpletransformers.classification import (
#     ClassificationArgs,
#     ClassificationModel,
# )

# sweep_config = {
#     "method": "bayes",  # grid, random
#     "metric": {"name": "train_loss", "goal": "minimize"},
#     "parameters": {
#         "num_train_epochs": {"values": [2, 3, 5]},
#         "learning_rate": {"min": 1e-5, "max": 4e-4},
#     },
# }

# sweep_id = wandb.sweep(sweep_config, project="Simple Sweep")

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Preparing train data
# train_df = train[['text', 'author_num']]
# train_df.columns = ["text", "author_num"]

# eval_df = test[['text', 'author_num']]
# eval_df.columns = ["text", "author_num"]

# model_args = ClassificationArgs()
# model_args.reprocess_input_data = True
# model_args.overwrite_output_dir = True
# model_args.evaluate_during_training = True
# model_args.manual_seed = 4
# model_args.labels_list = ["true", "false"]
# model_args.wandb_project = "Simple Sweep"

# def train_fn():
#     # Initialize a new wandb run
#     wandb.init()

#     # Create a TransformerModel
#     model = ClassificationModel(
#         'bert',
#         'bert-base-cased',
#         use_cuda=True,
#         args=model_args,
#         sweep_config=wandb.config,
#     )

#     # Train the model
#     model.train_model(train_df, eval_df=eval_df)

#     # Evaluate the model
#     model.eval_model(eval_df)

#     # Sync wandb
#     wandb.join()


# wandb.agent(sweep_id, train_fn)


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = train[['text', 'author_num']]
train_df.columns = ["text", "author_num"]

eval_df = test[['text', 'author_num']]
eval_df.columns = ["text", "author_num"]

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=5)
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluate_during_training_steps = 1000
model_args.overwrite_output_dir = True

# Create a ClassificationModel
model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=3,
    args=model_args,
) 

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
predictions, raw_outputs = model.predict(["Sam was a Wizard"])


In [ ]:
model.eval_model(eval_df)

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

softmax([ 6.859375  , -3.78320312, -4.09765625])

# Submission

In [ ]:
sample = pd.read_csv("/kaggle/input/spooky-author-identification/sample_submission.zip")
sample.head()

In [ ]:
test = pd.read_csv("/kaggle/input/spooky-author-identification/test.zip")
test

In [ ]:
test['text'].tolist()[0:3]

In [ ]:
logits = model.predict(test['text'].tolist())
scores = [softmax(l) for l in logits[1]]
scores[0:3]

In [ ]:
result=test
result["EAP"]=[sc[0] for sc in scores]
result["HPL"]=[sc[1] for sc in scores]
result["MWS"]=[sc[2] for sc in scores]
result.head()

In [ ]:
result = result[['id', 'EAP', 'HPL', 'MWS']]
result.to_csv("submission.csv", index=False)